# Scrapping Forebet Completed Matches - Predictions and Results of Matches

## Overview

This notebook is designed to scrape data from Forebet's prediction table for completed matches. The scraped data will be stored in a database and used to train our betting prediction model. The primary goals of this notebook are:

1. **Scrape Completed Matches Data**: Retrieve data for completed matches, including team names, win probabilities, score predictions, and actual match outcomes.
2. **Data Processing**: Clean and format the scraped data to ensure it is suitable for analysis and model training.
3. **Database Insertion**: Store the cleaned data in the `completed_matches` table within our database.
4. **Chrome Web Driver**: Make sure to download and install Chrome Webdriver.


## Requirements

- Python 3.x
- Libraries: `selenium`, `sklearn`, `pandas`, `sqlalchemy`, `re`
- pymysql installed
- MYSQL Workbench
- Access to the database where the `completed_matches` table and `training_data` is located

## Notes

- Ensure you have the correct database credentials set up before running this notebook.
- This notebook should be run weekly to keep the `completed_matches` table updated with the latest match results.
- training_data is a joint table of `completed_matches` and `previous_league_table_standings` from your DB.


### Setup and Imports:

In [ ]:
!pip install pandas
!pip install sqlalchemy
!pip install scikit-learn

In [ ]:
from selenium import webdriver
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.preprocessing import LabelEncoder
# from modules import team_labels_forebet

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# TEAM LABELS
team_labels_forebet = {
        'Arsenal': 1,
        'Aston Villa': 2,
        'Bournemouth': 3,
        'Brentford': 4,
        'Brighton': 5,
        'Burnley': 6,
        'Chelsea': 7,
        'Crystal Palace': 8,
        'Everton': 9,
        'Fulham': 10,
        'Ipswich Town':11,
        'Leeds United': 12,
        'Leicester City': 13,
        'Liverpool': 14,
        'Manchester City': 15,
        'Manchester United': 16,
        'Newcastle United': 17,
        'Norwich City': 18,
        'Sheffield United': 19,
        'Southampton': 20,
        'Tottenham': 21,
        'West Ham': 22,
        'Luton Town': 23,
        'Wolverhampton': 24,
        'Sheffield United': 25,
        'Nottingham Forest': 26,
    }

### Scraping Data:

In [ ]:
# Set up the Chrome driver
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Opens browser in full-screen
driver = webdriver.Chrome(options=chrome_options)

# Set the window size (optional, depends on your need)
driver.set_window_size(1920, 1080)

# Open the Forebet predictions page for the English Premier League
driver.get('https://www.forebet.com/en/football-tips-and-predictions-for-england/premier-league')

# Add a wait to ensure the element is present
try:
    # Wait for the consent pop-up and click the consent button
    consent_button = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.XPATH, "//p[@class='fc-button-label' and text()='Consent']"))
    )
    consent_button.click()
    print("Clicked the consent button!")
except Exception as e:
    print(f"Error: {e}")



In [ ]:
# Now find the fixture containers within the body element
match_fixture_containers = driver.find_elements("class name", "schema")
print(match_fixture_containers)

In [ ]:
# Initialize an empty list to store fixture details
fixtures_container = []


# Iterate through each match fixture container
for fixture in match_fixture_containers:
    # Extract the text from the fixture container
    fixture_text = fixture.text
    # Append the extracted text to the fixtures_container list
    fixtures_container.append(fixture_text)
    # Print the extracted fixture text
    print(fixture_text)


In [ ]:
# Closes the webdriver and forebet page
driver.quit()

In [ ]:
fixtures_container

### Data Cleaning and Processing

In [ ]:
# Luckily the data has \nEpl which we can use to separate epl matches from other leagues.
matches_data_cleaned_step_1 = [match.split("\nEPL") for match in fixtures_container]
epl_matches = matches_data_cleaned_step_1[0]
epl_matches

In [ ]:
# Extract the current weekly round from the first index in epl_matches
weekly_round = epl_matches[0]
# Split the string to get the number, and subtract by 1. Note Do not subtract by 1 if its the first week of the season
weekly_round = int(weekly_round.split(' ')[1])  - 1
weekly_round

#### Brief Notes:
The page separates Upcoming matches from Completed matches using 'Round'. We use this Round Number to confirm if we are getting the right data. Also i append it in the dataframe at the end of the notebook to help double check when working with aggregated data in the DataBase

In [ ]:
completed_matches = []

for match in epl_matches:
    if 'FT' in match:  # Check if 'FT' (full-time) is present in the match string
        completed_matches.append(match)

print("Completed Matches:")
print(completed_matches)

#### Now we can remove `PREVIEW` if only it exists

In [ ]:
# Remove '\nPRE\nVIEW' from completed matches
completed_matches = [match.replace('\nPRE\nVIEW', '') for match in completed_matches]

In [ ]:
print("\nCompleted Matches (after removing noise):\n")
print(completed_matches)

Note.

on 10/3/2024
i experienced a use case where the probabilities differ, in terms of length and expectation
usually we have double digits for home draw and away.
but in the case below Arsenal had a win probability of 55 and a draw of 38 leaving only room for a single digit to represent away win probability. This shifts the values hence previous regex operations do not pass this use case.

In [ ]:
'\nNewcastle United\nManchester City\n28/9/2024 12:30\n(22)(28)(50)(2)(1 - 3)3.2412°3/5 FT 1 - 1\n(0 - 1)', 
'\nArsenal\nLeicester City\n28/9/2024 15:00\n55)(38)(7)(1) (2 - 1)3.0214°1/5 FT 4 - 2\n(2 - 0)', 




In [ ]:
Solution
Hard Coding for now. Put zeros before the single 
['\nArsenal\nSouthampton\n5/10/2024 15:00\n7318913 - 03.4916°1/10\nPRE\nVIEW']
['\nWolverhampton\nLiverpool\n28/9/2024 17:30\n1388021 - 33.4812°2/5 FT 1 - 2\n(0 - 1)\n0.24']

In [ ]:
import re
from itertools import permutations

def insert_zero_and_validate(number_string):
    # Convert the string into a list of integers
    digits = [int(digit) for digit in number_string]
    
    # Check different groupings by inserting '0' in each possible spot
    for i in range(1, len(digits)):
        # Create a new list with a '0' inserted at position i
        new_digits = digits[:i] + [0] + digits[i:]
        
        # Now we use static indices for the grouping
        # Group digits as: (index 0 and 1), (index 2 and 3), (index 4 and 5)
        first_group = int("".join(map(str, new_digits[:2])))  # 1st and 2nd digits
        second_group = int("".join(map(str, new_digits[2:4])))  # 3rd and 4th digits
        third_group = int("".join(map(str, new_digits[4:6])))  # 5th and 6th digits
        
        # Sum of the three groups
        total = first_group + second_group + third_group
        
        # Check if the total is close to 100 (with margin of error of 1)
        if abs(100 - total) <= 1:
            return "".join(map(str, new_digits))
    
    return "No valid grouping found"
def extract_and_correct_probabilities(text):
    """
    This function extracts the probability portion of each match, corrects for any single digits, 
    and returns the updated probability string.
    """
    # Extract the portion before the dash
    prob_pattern = r'\n(\d+)\s-\s'  # Updated to capture 5 or 6 digit probabilities
    
    match = re.search(prob_pattern, text)
    if match:
        prob_str = match.group(1)  # The part containing probabilities
        
        if len(prob_str) < 8:
            # Handle the case where the probability string needs correction
            prob_str = prob_str[:-2]
            valid_probabilities = insert_zero_and_validate(prob_str)
            if valid_probabilities:
                # Replace the old probability string with the corrected one
                text = text.replace(prob_str, valid_probabilities)
    
    return text

def fractional_to_decimal(fractional):
    """
    Converts fractional odds to decimal odds.
    Example: '11/10' -> 2.10
    """
    try:
        numerator, denominator = map(int, fractional.split('/'))
        return round((numerator / denominator) + 1, 2)
    except Exception as e:
        print(f"Error converting fractional odds: {e}")
        return fractional

def replace_completed_matches(text):
    """
    This function applies the main regex to update the completed matches data.
    """
    def replacement_function(match):
        # Extract all groups from the regex
        group1 = match.group(1)
        group2 = match.group(2)
        group3 = match.group(3)
        group4 = match.group(4)
        group5 = match.group(5)
        group6 = match.group(6)
        group7 = match.group(7)
        decimal_odds = match.group(8)
        group9 = match.group(9)

        # If odds are in fractional format, convert them to decimal
        if '/' in decimal_odds:
            decimal_odds = fractional_to_decimal(decimal_odds)

        # Rebuild the string with updated decimal odds for the 8th group
        return f"\n{group1}\n{group2}\n{group3}\n{group4}\n{group5}\n{group6}\n{group7}\n{decimal_odds}\n{group9}"

    # Regular expression pattern for matching completed matches data, including both decimal and fractional odds.
    pattern_completed_matches = r'\n(\d{2})(\d{2})(\d{2})([A-Z]|\d?)(\d?\s-\s\d{1})(\d{1}.\d{2}|\d{1,2}\/\d{1,2})(\d{2}°|\d{1}°)(\d?.\d{2}|\d{1,2}\/\d{1,2})\s(FT\s\d?\s-\s\d?)'

    # Perform the replacement on the text
    return re.sub(pattern_completed_matches, replacement_function, text)


# Step-by-step processing
for i in range(len(completed_matches)):
    # Step 1: Correct probabilities
    completed_matches[i] = extract_and_correct_probabilities(completed_matches[i])
    
    # Step 2: Apply the main regex replacement
    completed_matches[i] = replace_completed_matches(completed_matches[i])

# Output the processed matches
for match in completed_matches:
    print(match)


### REGEX: Preprocessing Completed matches array 

# v1.2
After coming to the Uk, i realized forebet has different representations for odds. the uk system the american and the european, which africans also use.

In [ ]:
import re

# Regular expression pattern for matching completed matches data, including both decimal and fractional odds.
# Fractional odds are in the form a/b and need conversion
pattern_completed_matches = r'\n(\d{2})(\d{2})(\d{2})([A-Z]|\d?)(\d?\s-\s\d{1})(\d{1}.\d{2}|\d{1,2}\/\d{1,2})(\d{2}°|\d{1}°)(\d?.\d{2}|\d{1,2}\/\d{1,2})\s(FT\s\d?\s-\s\d?)'

def fractional_to_decimal(fractional):
    """
    Converts fractional odds to decimal odds.
    Example: '11/10' -> 2.10
    """
    try:
        numerator, denominator = map(int, fractional.split('/'))
        return round((numerator / denominator) + 1, 2)
    except Exception as e:
        print(f"Error converting fractional odds: {e}")
        return fractional

def replace_completed_matches(text):
    def replacement_function(match):
        # Extract all groups from the regex
        group1 = match.group(1)
        group2 = match.group(2)
        group3 = match.group(3)
        group4 = match.group(4)
        group5 = match.group(5)
        group6 = match.group(6)
        group7 = match.group(7)
        decimal_odds = match.group(8)
        group9 = match.group(9)

        # If odds are in fractional format, convert them to decimal
        if '/' in decimal_odds:
            decimal_odds = fractional_to_decimal(decimal_odds)

        # Rebuild the string with updated decimal odds for the 8th group
        return f"\n{group1}\n{group2}\n{group3}\n{group4}\n{group5}\n{group6}\n{group7}\n{decimal_odds}\n{group9}"

    # Perform the replacement on the text
    return re.sub(pattern_completed_matches, replacement_function, text)




# Process completed matches
for i in range(len(completed_matches)):
    completed_matches[i] = replace_completed_matches(completed_matches[i])


In [ ]:
print("\nCompleted Matches (after removing noise):\n")
print(completed_matches)

# v.1
old version for european system. Just leave it here for documentation

In [ ]:
## Regular expression pattern for matching completed matches data
## Example matched string: '\n15196521 - 33.1921°1.25 FT 0 - 4'
## This represents:
## 15 - Probability of home team win
## 19 - Probability of draw
## 52 - Probability of away team win
## 2 - Team to win (1 for home, X for draw, 2 for away)
## 1 - Home team score prediction
## 3 - Away team score prediction
## 3.19 - Average goals prediction
## 21° - Weather in degrees
## 1.25 - Odds
## FT 0 - 4 - Actual full-time score

pattern_completed_matches = r'\n(\d{2})(\d{2})(\d{2})([A-Z]|\d?)(\d?\s-\s\d{1})(\d{1}.\d{2})(\d{2}°|\d{1}°)(\d?.\d{2})\s(FT\s\d?\s-\s\d?)'

replacement_completed_matches = r'\n\1\n\2\n\3\n\4\n\5\n\6\n\7\n\8\n\9'

def replace_completed_matches(text):
    """
    Replace matches of the completed matches pattern in the given text
    with the specified replacement pattern.

    Args:
        text (str): The input text containing completed matches data.

    Returns:
        str: The text with replaced patterns.
    """
    return re.sub(pattern_completed_matches, replacement_completed_matches, text)

# Process completed matches
for i in range(len(completed_matches)):
    completed_matches[i] = replace_completed_matches(completed_matches[i])


In [ ]:
# print("\nCompleted Matches (after removing noise):\n")
# print(completed_matches)

#### Brief Notes:
- pattern_completed_matches: A regular expression pattern to match the completed matches data format. This pattern captures various parts of the match data, including win probabilities, team scores, average goals, weather, odds, and the actual scoreline.
- replacement_completed_matches: A replacement pattern to reformat the matched completed matches data by adding newline characters (\n) between the captured groups. This facilitates easier creation of a DataFrame by separating the future columns. 
eg  \n15196521 - 33.1921°1.25 FT 0 - 4  becomes  => \n15\n19\n65\n2\n1 - 3\n3.19\n21°\n1.25\nFT 0 - 4

#### Split by \n

In [ ]:
# New array to hold split Data 
completed_matches_split = []

for i in range(len(completed_matches)):
    completed_matches_split.append(completed_matches[i].split('\n'))
    

In [ ]:
print("\nCompleted Matches Split:\n")
print(completed_matches_split)


In [ ]:
df_columns_completed_matches  = ['', 'home', 'away', 'date_and_time', 'home_win_probability', 'draw_probability', 'away_win_probability', 'team_to_win_prediction', 'scoreline_prediction', 'average_goals_prediction', 'weather_in_degrees', 'odds', 'full_time_score', 'score_at_halftime', "kelly_criterion"]

In [ ]:
# Create DataFrame
df_completed_matches = pd.DataFrame(completed_matches_split, columns=df_columns_completed_matches)

# Drop first column
df_completed_matches = df_completed_matches.drop(columns=[''])

# Display DataFrame
print(df_completed_matches)

We are going to skip EDA since the data varies week in week out. Later when i connect the data to a database, and keep track of certain trends maybe we can notice certain patterns

### DataBase and Model Readiness: 
We are still preparing the data, but from now on, everything we do has the model in mind, that means label encoding, converting string to floats dropping columns etc. so its a bit different from just cleaning the data. its more like tuning ? parameter tuning ?

We are going to create labels for the teams in the premier league, in order to have Home and Away values are numerical labels. important for logisit regression. Might not be the best model, but yh 
Sometimes reading the teams labels in figures is annoying, so you can chose to skip this part, and only copy the code for label encoding when you are ready to train the model

In [ ]:
def team_to_label(team_name):
    """
    Convert a team name to its corresponding label.

    Args:
        team_name (str): The name of the team.

    Returns:
        int: The corresponding label for the team name as per the team_labels dictionary.
    """
    return team_labels_forebet.get(team_name)


In [ ]:
# Map home and away team names to corresponding labels.
df_completed_matches['home'] = df_completed_matches['home'].map(team_to_label)
df_completed_matches['away'] = df_completed_matches['away'].map(team_to_label)


print(df_completed_matches.head())

### Splitting Date and Time 

In [ ]:
df_completed_matches[['date', 'time']] = df_completed_matches['date_and_time'].str.split(' ', expand=True)

### Splitting the "Scoreline prediction" column into separate columns (Home goals, Away goals)

In [ ]:
df_completed_matches[['home_team_score_prediction', 'away_team_score_prediction']] = \
    df_completed_matches['scoreline_prediction'].str.split('-', expand=True)


# Converting the split columns to integers
df_completed_matches['home_team_score_prediction'] = df_completed_matches['home_team_score_prediction'].astype(int)
df_completed_matches['away_team_score_prediction'] = df_completed_matches['away_team_score_prediction'].astype(int)

# Example usage:
print(df_completed_matches.head())  # Display the first few rows to verify the changes


In [ ]:
df_completed_matches.drop(columns=['scoreline_prediction'], inplace=True)

### Splitting the "Halftime scoreline" column into separate columns (Home goals, Away goals)

In [ ]:
df_completed_matches[['home_team_full_time_score', 'away_team_full_time_score']] = df_completed_matches['full_time_score'].str.strip('FT ').str.split(' - ', expand=True)

df_completed_matches['away_team_full_time_score'] = df_completed_matches['away_team_full_time_score'].astype(int)
df_completed_matches['home_team_full_time_score'] = df_completed_matches['home_team_full_time_score'].astype(int)


In [ ]:

df_completed_matches[['home_team_halftime_score', 'away_team_halftime_score']] = df_completed_matches['score_at_halftime'].str.strip('()').str.split(' - ', expand=True)


### Creating Prediction win/loss Column 

In [ ]:
def create_y(df):
    """
    Create the target variable (y) based on the prediction results.

    Args:
        df (DataFrame): The DataFrame containing match predictions and actual outcomes.

    Returns:
        list: The target variable (y) indicating whether the prediction was correct (1) or not (0)
    """
    y = []
    for i in range(len(df)):
        if df['team_to_win_prediction'][i] == '1' and df['home_team_full_time_score'][i] > df['away_team_full_time_score'][i]:
            y.append(1)
        elif df['team_to_win_prediction'][i] == '2' and df['home_team_full_time_score'][i] < df['away_team_full_time_score'][i]:
            y.append(1)
        elif df['team_to_win_prediction'][i] == 'X' and df['home_team_full_time_score'][i] == df['away_team_full_time_score'][i]:
            y.append(1)
        else:
            y.append(0)
    return y

# Append the y column to the main DataFrame
df_completed_matches['prediction_result'] = create_y(df_completed_matches)


#### Brief Notes
This function iterates through each row in the DataFrame, checks the prediction result against the actual outcome, and assigns a value of 1 if the prediction was correct and 0 otherwise. This is what our model will be trained on. Can the model spot the patterns in predictions that are usually correct and affirm future predictions for upcoming matches ?

In [ ]:
print(df_completed_matches)

## Feature engineering

We will make sure our columns are in the right format and again all with the model in mind. 

In [ ]:
# df_completed_matches[['home', 'away', 'team_to_win_prediction','prediction_result']] = df_completed_matches[['home', 'away', 'team_to_win_prediction','prediction_result']].astype('category')

In [ ]:
# Convert probabilities to float
df_completed_matches['home_win_probability'] = df_completed_matches['home_win_probability'].astype(float)
df_completed_matches['draw_probability'] = df_completed_matches['draw_probability'].astype(float)
df_completed_matches['away_win_probability'] = df_completed_matches['away_win_probability'].astype(float)
df_completed_matches['average_goals_prediction'] = df_completed_matches['average_goals_prediction'].astype(float)
df_completed_matches['odds'] = df_completed_matches['odds'].astype(float)

In [ ]:
# Convert relevant score columns to integers
df_completed_matches['home_team_full_time_score'] = df_completed_matches['home_team_full_time_score'].astype(int)
df_completed_matches['away_team_full_time_score'] = df_completed_matches['away_team_full_time_score'].astype(int)
df_completed_matches['home_team_halftime_score'] = df_completed_matches['home_team_halftime_score'].astype(int)
df_completed_matches['away_team_halftime_score'] = df_completed_matches['away_team_halftime_score'].astype(int)


In [ ]:
df_completed_matches.drop(columns=['kelly_criterion'], inplace=True)

In [ ]:
# Depending on your needs, you can extract features from date and time columns
# For example, to extract day of the week and month from 'Date':
# Convert 'Date' column to datetime with correct format
df_completed_matches['date'] = pd.to_datetime(df_completed_matches['date'], format='%d/%m/%Y')
df_completed_matches['day_of_week'] = df_completed_matches['date'].dt.dayofweek
df_completed_matches['month'] = df_completed_matches['date'].dt.month
df_completed_matches['weekly_round'] = weekly_round
# Finally change the date_and_time column to Datetime format ? VERIFY
df_completed_matches['date_and_time'] = pd.to_datetime(df_completed_matches['date_and_time'], format='%d/%m/%Y %H:%M')
# Assuming the 'time' column is in the format '%H:%M' (24-hour clock)
df_completed_matches['time'] = pd.to_datetime(df_completed_matches['time'], format='%H:%M').dt.time


In [ ]:
# Custom label mapping
label_mapping = {'X': 0, '1': 1, '2': 2}
df_completed_matches['team_to_win_prediction'] = df_completed_matches['team_to_win_prediction'].map(label_mapping)

### Validation and Logging: 

In [ ]:
df_completed_matches.info()

In [ ]:
df_completed_matches["time"]

### PERSISTENT STORAGE

In [ ]:
!pip install pymysql

In [ ]:
# Database connection
user = 'test_user'
password = 'password'
host = 'localhost'
port = 3306
database = 'bet_prediction_model'

In [ ]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
df_completed_matches.to_sql('completed_matches', con=engine, if_exists='replace', index=False)